In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py

In [18]:
temp_country = pd.read_csv('data/bad_global_land_temperatures_by_country.csv')

In [19]:
#Removing the duplicated countries and countries for which no information about the temperature
temp_country_clean = temp_country[~temp_country['Country'].isin(
    ['Denmark', 'Antarctica', 'France', 'Europe', 'Netherlands', 'United Kingdom', 'Africa', 'South America'])]
temp_country_clean = temp_country_clean.replace(
    ['Denmark (Europe)', 'France (Europe)', 'Netherlands (Europe)', 'United Kingdom (Europe)'],
    ['Denmark', 'France', 'Netherlands', 'United Kingdom'])

In [20]:
temp_country_clean['dt'] = pd.to_datetime(temp_country_clean['dt'])

In [21]:
start_date = pd.to_datetime('1900-1-1')
end_date = pd.to_datetime('2013-1-1')
mask = temp_country_clean['dt'] >= start_date
temp_country_1900 = temp_country_clean.loc[mask]
temp_country_1900

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
1874,1900-01-01,-3.026,0.538,Åland
1875,1900-02-01,-8.063,0.657,Åland
1876,1900-03-01,-3.196,0.467,Åland
1877,1900-04-01,0.781,0.224,Åland
1878,1900-05-01,4.960,0.503,Åland
...,...,...,...,...
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe


In [22]:
temp_country_yearly = temp_country_1900.groupby([pd.Grouper(key='dt', freq='Y'), 'Country'])[
    'AverageTemperature'].mean().reset_index(name='Year')
temp_country_yearly.rename(columns={'dt': 'Year', 'Year': 'AverageTemperature'}, inplace=True)
temp_country_yearly['Year'] = pd.DatetimeIndex(temp_country_yearly['Year']).year
temp_country_yearly

,Year,Country,AverageTemperature
0,1900,Afghanistan,13.749333
1,1900,Albania,13.068583
2,1900,Algeria,22.864167
3,1900,American Samoa,26.273500
4,1900,Andorra,11.348333
...,...,...,...
26683,2013,Western Sahara,23.744250
26684,2013,Yemen,28.129750
26685,2013,Zambia,21.196000
26686,2013,Zimbabwe,20.710750


In [23]:
countries = np.unique(temp_country_yearly['Country'])
temp_dif_lst = []
for country in countries:
    country_temp = temp_country_yearly[(temp_country_yearly['Country'] == country) & (
                (temp_country_yearly['Year'] == 2013) | (temp_country_yearly['Year'] == 1900))][
        'AverageTemperature'].diff()
    country_temp.drop(index=country_temp.index[0], axis=0,
                      inplace=True)
    temp_dif_country = country_temp.values
    if len(temp_dif_country) > 0:
        temp_dif_lst.append(temp_dif_country[0])


In [24]:
data = [dict(type='choropleth', locations=countries, z=temp_dif_lst, locationmode='country names', text=countries,
             marker=dict(line=dict(color='rgb(0,0,0)', width=1)),
             colorbar=dict(autotick=True, tickprefix='', title='# Temperature \nDifferences,\n°C'))]
layout = dict(title='Temperature differences in countries (1900 - 2013)',
              geo=dict(showframe=False, showocean=True, oceancolor='rgb(0,255,255)',
                       projection=dict(type='orthographic', rotation=dict(lon=60, lat=10), ),
                       lonaxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)'),
                       lataxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)')), )
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='worldmap')

In [25]:
emission = pd.read_csv('data/bad_historical_emission_by_country.csv')
emission

,Country,Data source,Sector,Gas,Unit,2018,2017,2016,2015,2014,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,World,CAIT,Total including LUCF,CO2,MtCO₂e,36441.55,35588.70,35160.60,34521.91,34558.59,...,24957.30,24895.32,25292.92,24214.92,23890.22,23260.29,23124.00,22988.29,23056.40,22849.92
1,World,CAIT,Total excluding LUCF,CO2,MtCO₂e,35248.74,34542.44,34081.00,34038.49,34098.75,...,23398.99,23240.47,23082.12,22694.41,22215.88,21591.78,21455.52,21319.81,21387.92,21181.44
2,World,CAIT,Energy,CO2,MtCO₂e,33746.74,33070.44,32620.00,32608.49,32627.75,...,22714.09,22584.68,22423.32,22054.72,21589.48,20999.68,20899.32,20783.81,20873.62,20680.14
3,World,CAIT,Electricity/Heat,CO2,MtCO₂e,15590.95,15167.36,14949.51,15009.83,15232.52,...,10134.66,10080.66,9853.19,9586.39,9231.22,9042.50,8945.80,8894.60,8735.04,8596.17
4,China,CAIT,Total excluding LUCF,CO2,MtCO₂e,10313.46,10017.77,9814.31,9830.43,9894.94,...,3149.20,3232.12,3128.65,3064.88,3084.87,2763.90,2643.53,2418.18,2302.19,2173.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,Saint Vincent and the Grenadines,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2303,Vanuatu,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2304,Samoa,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2305,Zambia,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [26]:
emission['Mean'] = emission.iloc[:, 5:34].mean(axis=1).round(decimals=3)
total_emission = emission.iloc[1:,:][emission['Sector'] == 'Total including LUCF']
total_emission

/var/folders/st/5wgrjpj9129bgxw_hskwwwlh0000gn/T/ipykernel_94554/1476003398.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Country,Data source,Sector,Gas,Unit,2018,2017,2016,2015,2014,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,Mean
5,China,CAIT,Total including LUCF,CO2,MtCO₂e,9663.36,9367.67,9164.21,9120.27,9184.77,...,2882.75,2779.27,2715.50,2735.48,2414.50,2294.12,2068.77,1952.78,1823.96,5405.082
12,United States,CAIT,Total including LUCF,CO2,MtCO₂e,4749.57,4581.90,4656.84,4563.52,4683.35,...,5172.06,5129.29,4864.46,4708.31,4654.52,4581.76,4461.62,4389.50,4426.40,4900.768
17,European Union (27),CAIT,Total including LUCF,CO2,MtCO₂e,2636.99,2692.12,2669.54,2321.61,2263.78,...,3134.03,3142.23,3210.93,3113.68,3060.09,3072.40,3133.06,3247.48,3286.44,2946.370
19,India,CAIT,Total including LUCF,CO2,MtCO₂e,2400.25,2267.16,2149.01,2085.38,2072.03,...,618.73,600.38,555.60,519.98,466.79,431.31,409.09,386.17,341.32,1192.656
27,Indonesia,CAIT,Total including LUCF,CO2,MtCO₂e,1269.55,1081.11,1068.10,1574.76,1539.23,...,907.96,1484.56,772.38,909.21,881.31,869.88,855.30,846.23,832.39,1002.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,Nauru,CAIT,Total including LUCF,CO2,MtCO₂e,0.07,0.07,0.06,0.05,0.05,...,0.10,0.10,0.10,0.11,0.11,0.11,0.12,0.13,0.13,0.076
1934,Tuvalu,CAIT,Total including LUCF,CO2,MtCO₂e,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.009
2041,Niue,CAIT,Total including LUCF,CO2,MtCO₂e,0.00,0.00,0.00,0.00,0.00,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.007
2125,Fiji,CAIT,Total including LUCF,CO2,MtCO₂e,-0.65,-0.73,-0.81,-1.03,-1.24,...,-1.76,-1.74,-1.72,-1.78,-1.79,-1.79,-1.78,-1.81,-1.81,-1.454


In [27]:
countries_emission = np.unique(total_emission['Country'])
mean_emission_lst = []
for country in countries_emission:
    mean_total_emission = total_emission[(total_emission['Country'] == country)]['Mean']
    if len(mean_total_emission.values) > 0:
        mean_emission_lst.append(mean_total_emission.values[0])

In [28]:
emission_countries_yearly = total_emission.melt(id_vars=["Country"], value_vars=total_emission.iloc[:,5:34], var_name="Year", value_name="TotalEmission")

In [29]:
data_emission = [dict(type='choropleth', locations=countries_emission, z=mean_emission_lst, locationmode='country names', text='Mean emission',
             marker=dict(line=dict(color='rgb(0,0,0)', width=1)),
             colorbar=dict(autotick=True, tickprefix='', title='Mean\nEmission,\n(MtCO₂e)'))]
layout = dict(title='Mean CO2 emission in countries (1990 - 2018)',
              geo=dict(showframe=False, showocean=True, oceancolor='rgb(0,255,255)',
                       projection=dict(type='orthographic', rotation=dict(lon=60, lat=10), ),
                       lonaxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)'),
                       lataxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)')), )
fig_emission = dict(data=data_emission, layout=layout)
py.iplot(fig_emission, validate=False, filename='worldmap')


In [30]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

us = temp_country_yearly[temp_country_yearly.Country == "United States"]
us_emission = emission_countries_yearly[emission_countries_yearly["Country"] == "United States"]

fig.add_trace(go.Scatter(x=us.Year, y=us.AverageTemperature, name="Average Temperature"), secondary_y=False)

fig.add_trace(go.Scatter(x=us_emission.Year, y=us_emission.TotalEmission, name="Total Emission"), secondary_y=True)

fig.update_layout(title_text="Average Temperature and Total Emission of the United States")

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Temperature", secondary_y=False)
fig.update_yaxes(title_text="Total Emission", secondary_y=True)

fig.show()